In [1]:
import xarray as xa
import math
import numpy as np
import datetime as dt

# Mapping degree day accumulation per time period $\Delta t$.

Here's the weather dataset from 1950-2015:

In [2]:
weather = xa.open_mfdataset("weatherdata/*.nc", decode_times=False, chunks={'time':50});
print weather

<xarray.Dataset>
Dimensions:    (latitude: 73, longitude: 96, time: 23739)
Coordinates:
  * latitude   (latitude) float32 90.0 87.5 85.0 82.5 80.0 77.5 75.0 72.5 ...
  * longitude  (longitude) float32 0.0 3.75 7.5 11.25 15.0 18.75 22.5 26.25 ...
  * time       (time) float64 7.122e+05 7.122e+05 7.122e+05 7.122e+05 ...
Data variables:
    tmax       (time, latitude, longitude) float32 nan nan nan nan nan nan ...
    tmin       (time, latitude, longitude) float32 nan nan nan nan nan nan ...


Next we'll implement the degree days function, implemented [here](https://github.com/lbuckley/ICBseasonality/blob/master/DDFunctions.R) in R. 

In [3]:
## coords: NP series containing:
##    tmin: temperature minimum value
##    tmax: temperature max value
## ldt: lower developmental threshold.
def degreeDays(coords, ldt): 
    tmin = float(coords.tmin)
    tmax = float(coords.tmax)
    ldt  = float(ldt)
    
    if (math.isnan(tmin) or math.isnan(tmax)): return -1
    
    degreeDays = 0.0
    
    if (tmin >= ldt): # temperature entirely above ldt
        degreeDays = (tmax + tmin)/2-ldt
    
    if (tmin < ldt and tmax > ldt):
        alpha = (tmax-tmin)/2
        theta1 = math.asin(((ldt - (tmax+tmin))/alpha)*math.pi/180.0)
        degreeDays = 1/math.pi*(((tmax+tmin)/2-ldt)*(math.pi/2-theta1)+alpha*math.cos(theta1))
        
        if(degreeDays < 0):
            degreeDays = 0
        
    if (tmax <= ldt):
        degreeDays = 0
        
    return degreeDays
        
\

In [ ]:
df = weather.sel(latitude=30, longitude=-123.0, method='nearest').to_dataframe()
df_range = weather.sel(latitude=slice(30,20)).to_dataframe()

In [ ]:
df_range.apply(degreeDays, axis=1, ldt=10)

In [ ]:
newdim = xa.concat([tmax, tmin], 'concat').where((latitude > 20 & longitude < -120))